In [ ]:
import pandas as pd
import holoviews as hv
import hvplot.pandas

from seastats.stats import get_stats

## load files 

In [ ]:
sim = pd.read_parquet('../tests/data/abed_sim.parquet')
obs = pd.read_parquet('../tests/data/abed_obs.parquet')

In [ ]:
# sim and obs need to be Series
obs = obs[obs.columns[0]]
sim = sim[sim.columns[0]]

In [ ]:
obs.hvplot(line_dash = "dashed") * sim.hvplot()

## Basic metrics

In [ ]:
stats = get_stats(obs, sim)
stats

## storm metrics

we defined the following metrics for the storms events: 

* `R1`/`R3`/`error_metric`: we select the biggest observated storms, and then calculate error (so the absolute value of differenc between the model and the observed peaks)
  * `R1` is the error for the biggest storm
  * `R3` is the mean error for the 3 biggest storms
  * `error_metric` is the mean error for all the storms above the threshold.

* `R1_norm`/`R3_norm`/`error`: Same methodology, but values are in normalised (in %) by the observed peaks.

In [ ]:
from seastats.storms import storm_metrics
from pyextremes import get_extremes

quantile = 0.99
cluster_duration = 72

In [ ]:
# storm metrics
metric99 = storm_metrics(sim, obs, quantile=quantile, cluster_duration=cluster_duration)
metric99

In [ ]:
threshold = sim.quantile(quantile)
ext_ = get_extremes(sim, "POT", threshold=threshold, r=f"{cluster_duration}h")
extremes1 = pd.DataFrame({"modeled" : ext_, "time_model" : ext_.index}, index=ext_.index)

In [ ]:
storm_plot = sim.hvplot() * hv.Scatter((extremes1.index, extremes1.modeled)).opts(color="red")
storm_plot

In [ ]:
threshold = obs.quantile(quantile)
ext_ = get_extremes(obs, "POT", threshold=threshold, r=f"{cluster_duration}h")
extremes2 = pd.DataFrame({"modeled" : ext_, "time_model" : ext_.index}, index=ext_.index)

In [ ]:
storm_plot = obs.hvplot(line_dash = "dashed") * hv.Scatter((extremes2.index, extremes2.modeled)).opts(color="red")
storm_plot

complete the stats dictionary

In [ ]:
metric99 = storm_metrics(sim, obs, quantile=0.99, cluster_duration=72)
metric95 = storm_metrics(sim, obs, quantile=0.95, cluster_duration=72)
# Create a dictionary for the current version's statistics
stats["R1"] = metric99["R1"]
stats["R1_norm"] = metric99["R1_norm"]
stats["R3"] = metric99["R3"]
stats["R3_norm"] = metric99["R3_norm"]
stats["error99"] = metric99["error"]
stats["error99m"] = metric99["error_metric"]
stats["error95"] = metric95["error"]
stats["error95m"] = metric95["error_metric"]
stats